In [10]:
##Class: ANLY-580_03
##Professor: Trevor Adriaanse
##Topic: Therapy Chatbot
##Team Member: Xinran Zhang, Qian Chen, Ting Huang, Yifan(Jake) Zhu

In [11]:
#Load necessary packages
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import json


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [12]:
# Splits each line of csv file to create conversation_object and conversations
def loadConversations(filepath):

    conversations = {}
    with open(filepath) as csvfile:
        reader = csv.DictReader(csvfile)
        id_index = 1
        for row in reader:
            conversation_object = {}
            conversation_object['id'] = id_index
            conversation_object['tag'] = row['tag']
            conversation_object['pattern'] = row['pattern']
            conversation_object['response'] = row['response']

            conversations[conversation_object['id']] = conversation_object

            id_index +=1

    return conversations


# Extracts pairs of pattern and response from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations.values():

        # Iterate over each conversation and retrieve pattern and response
        pattern = conversation["pattern"].strip()
        response = conversation["response"].strip()
        qa_pairs.append([pattern, response])

    return qa_pairs

In [13]:
# Define path to new file with formatted data
datafile = os.path.join('data/formatted_lines.txt')

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Initialize conversations dict
conversations = {}
# Load conversations
conversations = loadConversations('data/final.csv')

# Write new file
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

In [14]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self):
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [15]:
MAX_LENGTH = 20  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read pattern/response pairs and return a voc object
def readVocs(datafile):
    print("Reading lines...")
    # Read the file and split into pairs
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc()
    return voc, pairs

# Returns True if both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(datafile):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Load/Assemble voc and pairs
# save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(datafile)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 700 sentence pairs
Trimmed to 666 sentence pairs
Counting words...
Counted words: 770

pairs:
['hi', 'hello there . tell me how are you feeling today ?']
['hey', 'hi there . what brings you here today ?']
['is anyone there', 'hi there . how are you feeling today ?']
['hi there', 'great to see you . how do you feel currently ?']
['hey there', 'how do you do ? do you have a good day ?']
['howdy', 'nice to meet you sir']
['hola', 'good to see you sir']
['hiya', 'hey ! long time no see . how are you ?']
['yo', 'what s the story ?']
['hey chatbot', 'it s a pleasure to meet you']


In [17]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[ 65,  65,  65, 155,  16],
        [641, 624,  25,  11,  10],
        [665,  65, 146, 600,  11],
        [ 54, 234, 198, 126, 206],
        [667,  91, 406, 614,  14],
        [592, 235,   6,  14,   2],
        [ 76,   6,   2,   2,   0],
        [636,   2,   0,   0,   0],
        [  8,   0,   0,   0,   0],
        [  6,   0,   0,   0,   0],
        [  2,   0,   0,   0,   0]])
lengths: tensor([11,  8,  7,  7,  6])
target_variable: tensor([[675, 100,  46,  65,  65],
        [  6, 155,  42,  86, 116],
        [ 19,  11, 100, 534, 149],
        [  5,   7, 227, 126, 186],
        [217,  43,  11, 180,   6],
        [ 65, 235, 165,  22, 172],
        [ 72, 122, 166, 600,  42],
        [ 24,  16,  25,  51, 534],
        [ 22, 201, 218, 193, 141],
        [636,  14, 407,   6,  84],
        [ 14,   2, 314,   2,   2],
        [  2,   0,  13,   0,   0],
        [  0,   0,   6,   0,   0],
        [  0,   0,   2,   0,   0]])
mask: tensor([[ True,  True,  True,  True,  True],
 